# Setup

Note: you might need to run `Pkg.instantiate()` to ensure that the `Manifest.toml` is up to date. This only needs to be done once.

In [1]:
import Pkg;
Pkg.activate(@__DIR__)
Pkg.status()

  Activating project at `/global/u1/b/blaschke/juliacon24-hpcworkshop/parts/mpi/explanation`


Status `/global/u1/b/blaschke/juliacon24-hpcworkshop/parts/mpi/explanation/Project.toml`
  [1520ce14] AbstractTrees v0.4.5
  [052768ef] CUDA v5.4.2
  [adafc99b] CpuId v0.3.1
  [0e44f5e4] Hwloc v3.0.1
  [da04e1cc] MPI v0.20.20
  [e7922434] MPIClusterManagers v0.2.4
  [6f74fd91] NetworkInterfaceControllers v0.1.0


In [2]:
using MPI

using NetworkInterfaceControllers, Sockets
interfaces = NetworkInterfaceControllers.get_interface_data(IPv4)

hsn0_public = filter(x->(x.name=="hsn0:chn" && x.version==:v4), interfaces) |> only 
public_slingshot_name = getnameinfo(hsn0_public.ip)

# to import MPIManager
using MPIClusterManagers

# need to also import Distributed to use addprocs()
using Distributed

# specify, number of mpi workers, launch cmd, etc.
manager=MPIWorkerManager(4)

# start mpi workers and add them as julia workers too.
addprocs(
    manager,
    exeflags=`--project=$(Base.active_project())`,
    master_tcp_interface=public_slingshot_name
)

4-element Vector{Int64}:
 2
 3
 4
 5

# Communication with MPI.jl

Picking up from the previous demo, we have a job with 4 ranks: 

In [3]:
@mpi_do manager begin
    using MPI: MPI, Comm, Win, free
    comm = MPI.COMM_WORLD
    rank = MPI.Comm_rank(comm)
    size = MPI.Comm_size(comm)
    name = gethostname()
    println("Hello world, I am $(rank) of $(size) on $(name)")
end

      From worker 5:	Hello world, I am 3 of 4 on nid200349
      From worker 2:	Hello world, I am 0 of 4 on nid200344
      From worker 4:	Hello world, I am 2 of 4 on nid200348
      From worker 3:	Hello world, I am 1 of 4 on nid200345


## Domain Decomposition

PDE solvers often break up work over a "grid" of ranks (domain decomposition). This will find the dimension of this grid:

In [4]:
@mpi_do manager begin
    dims = [0]
    MPI.Dims_create!(size, dims)
end

In [5]:
@mpi_do manager begin
    println(dims)
end

      From worker 3:	[4]
      From worker 2:	[4]
      From worker 4:	[4]
      From worker 5:	[4]


Each rank has the same value for `dims`. In $N$-dimensions, `length(dims) == N`.

## Cartesian Grids

We will now lay out each rank in a "grid" (in this example, $N=1$ so it's actually a line). In the excercise, $N=2$, so this will be an actual "grid". The steps here are pretty much the same though.

In [6]:
@mpi_do manager begin
    comm_cart = MPI.Cart_create(
        comm,  # MPI Communicator
        dims,  # Dimensions of grid
        [0],   # 0 == not periodic, 1 == periodic
        1,     # 0 == not allowed to reorder, 1 == allowed to reoder
    )
    me        = MPI.Comm_rank(comm_cart)
    coords    = MPI.Cart_coords(comm_cart)
    neighbors = MPI.Cart_shift(
        comm_cart,
        0,  # Which dimension to shift (zero-indexed)
        1,  # Shift magnitude
    )
end

In [7]:
@mpi_do manager begin
    comm_cart = MPI.Cart_create(
        comm,  # MPI Communicator
        dims,  # Dimensions of grid
        [0],   # 0 == not periodic, 1 == periodic
        1,     # 0 == not allowed to reorder, 1 == allowed to reoder
    )
    me        = MPI.Comm_rank(comm_cart)
    coords    = MPI.Cart_coords(comm_cart)
    neighbors = MPI.Cart_shift(
        comm_cart,
        0,  # Which dimension to shift (zero-indexed)
        1,  # Shift magnitude
    )
end

In [8]:
@mpi_do manager begin
    println("rank=$(me); coord=$(coords), neighbors=$(neighbors)")
end

      From worker 2:	rank=0; coord=[0], neighbors=(-1, 1)
      From worker 3:	rank=1; coord=[1], neighbors=(0, 2)
      From worker 5:	rank=3; coord=[3], neighbors=(2, -1)
      From worker 4:	rank=2; coord=[2], neighbors=(1, 3)


In [9]:
@mpi_do manager begin
    println("rank=$(me); coord=$(coords), neighbors=$(neighbors)")
end

      From worker 5:	rank=3; coord=[3], neighbors=(2, -1)
      From worker 2:	rank=0; coord=[0], neighbors=(-1, 1)
      From worker 4:	rank=2; coord=[2], neighbors=(1, 3)
      From worker 3:	rank=1; coord=[1], neighbors=(0, 2)


MPI contains several constants, for example what `-1` means in the context above. This means that there is "no neighbor" there:

In [10]:
MPI.PROC_NULL

-1

## Point-to-point Communication

Let's do something harder:
1. Each rank draws a random number between 1 and 100
2. Each rank's random number is shared with its neighbors

This is an example of point-to-point communication on a grid. We'll be using the same communication pattern in the excercise.

First we generate a andom number on each rank

In [11]:
@mpi_do manager begin
    using Random
    my_int = rand(1:100)
end

In [12]:
@mpi_do manager begin
    println("rank=$(me); my_int=$(my_int)")
end

      From worker 2:	rank=0; my_int=38
      From worker 4:	rank=2; my_int=29
      From worker 5:	rank=3; my_int=70
      From worker 3:	rank=1; my_int=71


MPI uses zero-copy memory access => we need to set up buffers (arrays) to send and receive data.

In [13]:
@mpi_do manager begin
    send_1 = zeros(Int64, 1)
    send_2 = zeros(Int64, 1)
    recv_1 = zeros(Int64, 1)
    recv_2 = zeros(Int64, 1)
end

Now we fill the buffers by copying out data into it -- wherever a buffer is needed.

In [14]:
@mpi_do manager begin
    if neighbors[1] != MPI.PROC_NULL
        copyto!(send_1, my_int)
    end
    if neighbors[2] != MPI.PROC_NULL
        copyto!(send_2, my_int)
    end 
end

Now we're ready to perform a data transfer with MPI. MPI is (largely) transaction based. There is a receiving end, and a sending end. In order for a send to be successful, the receiver must be ready to receive.

To help coordinate all of this, we set up a request store:

In [15]:
@mpi_do manager begin
    reqs = MPI.MultiRequest(4)
end

And we transfer the data using non-blocking MPI communivation (`Isend` and `Irecv`). Pro tip: initiate receive before send

In [16]:
@mpi_do manager begin
    # Initiate data reciever
    if neighbors[1] != MPI.PROC_NULL
        MPI.Irecv!(recv_1, comm_cart, reqs[1]; source=neighbors[1])
    end
    if neighbors[2] != MPI.PROC_NULL
        MPI.Irecv!(recv_2, comm_cart, reqs[2]; source=neighbors[2])
    end
    # Send data
    if neighbors[1] != MPI.PROC_NULL
        MPI.Isend(send_1, comm_cart, reqs[3]; dest=neighbors[1])
    end
    if neighbors[2] != MPI.PROC_NULL
        MPI.Isend(send_2, comm_cart, reqs[4]; dest=neighbors[2])
    end
end

Notice how we tagged data with `source` and `dest`. This makes sure that data is received in the correct order (the middle ranks receive data from _both_ sides), and -- in the case of `Isend` -- that the data is sent to the correct rank.

When using non-blocking communication, it's good to wait for all transactions to be completed before using the buffers:

In [17]:
@mpi_do manager begin
    # Wait for all requests to finish
    MPI.Waitall(reqs)
end

Let's take a look at what we've transferred:

In [19]:
@mpi_do manager begin
    println(
        "rank=$(me); " *
        "my_int=$(my_int); prev=$(recv_1); next=$(recv_2)"
    )
end

      From worker 4:	rank=2; my_int=29; prev=[71]; next=[70]
      From worker 2:	rank=0; my_int=38; prev=[0]; next=[71]
      From worker 5:	rank=3; my_int=70; prev=[29]; next=[0]
      From worker 3:	rank=1; my_int=71; prev=[38]; next=[29]


KeyError: KeyError: key "usage_request" not found

KeyError: KeyError: key "usage_request" not found

KeyError: KeyError: key "usage_request" not found